# Querying external database sources of interest

* Enable users to integrate data from external databases of interest within BBP KG
* While using the Nexus Forge interface and BMO vocabulary as much as possible as
* While benefiting from out of the box (meta)data transformation to make them ready for BBP internal pipelines and applications
* Demo with Mouselight, NeuroElectro, UniProt

In [1]:
import os
import json

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.resources import Dataset

In [2]:
# import getpass
# TOKEN = getpass.getpass()

In [3]:
endpoint = "https://staging.nise.bbp.epfl.ch/nexus/v1"
BUCKET = "dke/kgforge"
forge = KnowledgeGraphForge("../../configurations/database-sources/prod-nexus-sources.yml", endpoint=endpoint, bucket=BUCKET, debug=True)

# List of Data sources

In [4]:
forge.db_sources(pretty=True)

Available Database sources:
UniProt
NeuroElectro
NeuroMorpho


In [5]:
sources = forge.db_sources()

In [6]:

data = {
       'origin': 'store',
       'source': 'DemoStore',
       'model': { 
          'name': 'DemoModel',
          'origin': 'directory',
          'source': "../../../tests/data/demo-model/" 
        }
}


In [7]:
from kgforge.specializations.databases import StoreDatabase
ds = StoreDatabase(forge, name="DemoDB", **data)

In [8]:
forge.add_db_source(ds)

In [9]:
forge.db_sources(pretty=True)

Available Database sources:
UniProt
NeuroElectro
NeuroMorpho
DemoDB


# Data source metadata

In [10]:
neuroelectro = sources['NeuroElectro']

## Get data mappings (hold transformations logic) per data type

* Data mappings are used to transform results obtained from the external data sources so that they are ready for consumption by BBP tools
* Perform automatic ontology linking

In [11]:
forge.mappings(source="NeuroElectro")

Managed mappings for the data source per entity type and mapping type:
   - ElectrophysiologicalFeatureAnnotation:
        * DictionaryMapping
   - ParameterAnnotation:
        * DictionaryMapping
   - ParameterBody:
        * DictionaryMapping
   - ScholarlyArticle:
        * DictionaryMapping
   - SeriesBody:
        * DictionaryMapping


In [12]:
forge.mappings('UniProt')

Managed mappings for the data source per entity type and mapping type:
   - Gene:
        * DictionaryMapping
   - Protein:
        * DictionaryMapping


In [13]:
from kgforge.specializations.mappings import DictionaryMapping
mapping = forge.mapping(entity="ScholarlyArticle", source="NeuroElectro")

In [14]:
print(mapping)

{
    id: forge.format("identifier", "scholarlyarticles", x.id)
    type:
    [
        Entity
        ScholarlyArticle
    ]
    abstract: x.abstract
    author: x.authors_shaped
    datePublished: x.date_issued
    identifier: x.identifiers
    isPartOf:
    {
        type: Periodical
        issn: x.issn
        name: x.journal
        publisher: x.publisher
    }
    name: f"article_{x.id}"
    sameAs: x.full_text_link
    title: x.title
    url: x.full_text_link
}


In [15]:
forge.db_sources(type_='Gene', pretty=True)

Available Database sources:
UniProt


# Search and Access data from data source

* Mapping are automatically applied to search results
* takes a mn for now => working on making it faster 

In [16]:
filters = {"type":"ScholarlyArticle"}
#map=True, use_cache=True, # download=True
resources = forge.search(filters, db_source="NeuroElectro", limit=2, debug=True) 
# Add function for checking datsource health => reqsuire health url from db


Submitted query:
   PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
   PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
   PREFIX commonshapes: <https://neuroshapes.org/commons/>
   PREFIX datashapes: <https://neuroshapes.org/dash/>
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
   PREFIX oa: <http://www.w3.org/ns/oa#>
   PREFIX obo: <http://purl.obolibrary.org/obo/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFI

In [17]:
len(resources)

2

In [18]:
print(resources[0])

{
    context: https://bbp.neuroshapes.org
    id: https://bbp.epfl.ch/neurosciencegraph/data/scholarlyarticles/34151
    type:
    [
        Entity
        ScholarlyArticle
    ]
    abstract: This study examined the passive membrane and action potential properties of serotonergic and nonserotonergic neurons in the rostral ventromedial medulla (RVM) of the rat using whole cell patch-clamp recording techniques in the slice. Serotonergic neurons were identified by immunoreactivity for tryptophan hydroxylase (TrpH). Spinally projecting neurons were retrogradely labeled with 1'-dioactadecyl-3,3,3',3'-tetramethylindocarbodyanine perchlorate (DiI). Three types of neurons were identified within both spinally projecting serotonergic and nonserotonergic populations. Type 1 neurons exhibited irregular or sporadic spontaneous activity interspersed with periods of quiescence. Type 2 neurons were not spontaneously active and were additionally discriminated by a more negative resting membrane poten

In [19]:
uquery = """
PREFIX up: <http://purl.uniprot.org/core/>
SELECT ?protein
WHERE {
  ?protein a up:Protein ;
  up:reviewed true.
}
"""

In [20]:
uresources = forge.sparql(query=uquery, db_source='UniProt', limit=10, debug=True)

Submitted query:
   
   PREFIX up: <http://purl.uniprot.org/core/>
   SELECT ?protein
   WHERE {
     ?protein a up:Protein ;
     up:reviewed true.
   }
     LIMIT 10



In [21]:
len(uresources)

10

In [22]:
uresources[0]

Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, _inner_sync=False, protein='http://purl.uniprot.org/uniprot/A0B137')

## Use Filters to search

In [23]:
from kgforge.core.wrappings.paths import Filter, FilterOperator

In [24]:

proteins = forge.search({'type': 'Protein', 'up:reviewed': True}, db_source='UniProt', limit=10, debug=True)

Submitted query:
   PREFIX up: <http://purl.uniprot.org/core/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX owl2xml: <http://www.w3.org/2006/12/owl2-xml#>
   PREFIX swrlb: <http://www.w3.org/2003/11/swrlb#>
   PREFIX protege: <http://protege.stanford.edu/plugins/owl/protege#>
   PREFIX swrl: <http://www.w3.org/2003/11/swrl#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX dc11: <http://purl.org/dc/terms/>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX foaf: <http://xmlns.com/foaf/0.1/>
   SELECT ?id WHERE {?id rdf:type up:Protein;
    up:reviewed ?v1 . 
    FILTER(?v1 = 'true'^^xsd:boolean)
   }  LIMIT 10



In [25]:
proteins[0]

Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, id='http://purl.uniprot.org/uniprot/A0B137', _inner_sync=False)

# Map resources

In [26]:
uniprot = sources['UniProt']

In [27]:
complete_query = """
PREFIX up: <http://purl.uniprot.org/core/>
SELECT ?id ?gene ?label ?subject ?gene_label
WHERE {
  ?id a up:Protein ;
  up:reviewed true ;
  up:encodedBy ?gene ;
  up:recommendedName / up:fullName ?label ;
  up:organism / up:scientificName ?subject .
  ?gene skos:prefLabel ?gene_label . 
}
"""

In [28]:
raw_proteins = uniprot.sparql(complete_query)

In [29]:
new_resource = uniprot.map(raw_proteins[0], 'Protein')

In [30]:

print(json.dumps(forge.as_jsonld(new_resource), indent=4))

[
    {
        "@context": "https://bbp.neuroshapes.org",
        "@id": "https://bbp.epfl.ch/neurosciencegraph/data/proteins/P0DJN9",
        "@type": [
            "Entity",
            "Protein"
        ],
        "encodedBy": {
            "@id": "http://purl.uniprot.org/uniprot/P0DJN9#gene-MD5A00DD99270221B359AB0AE338E423668",
            "label": "acsF"
        },
        "identifier": {
            "propertyID": "UniProtKB",
            "value": "P0DJN9"
        },
        "name": "Protein P0DJN9 from UniProtKB",
        "label": "Aerobic magnesium-protoporphyrin IX monomethyl ester [oxidative] cyclase",
        "subject": {
            "label": "Rubrivivax gelatinosus"
        }
    }
]


### same result could be obtain from a dictionary and a DictionaryMapping instance

In [31]:
dict_resource = forge.as_json(raw_proteins[0])
mapping = DictionaryMapping.load("../../database-sources/UniProt/mappings/DictionaryMapping/Protein.hjson")
print(json.dumps(forge.as_jsonld(uniprot.map(dict_resource, mapping)), indent=4))

[
    {
        "@context": "https://bbp.neuroshapes.org",
        "@id": "https://bbp.epfl.ch/neurosciencegraph/data/proteins/P0DJN9",
        "@type": [
            "Entity",
            "Protein"
        ],
        "encodedBy": {
            "@id": "http://purl.uniprot.org/uniprot/P0DJN9#gene-MD5A00DD99270221B359AB0AE338E423668",
            "label": "acsF"
        },
        "identifier": {
            "propertyID": "UniProtKB",
            "value": "P0DJN9"
        },
        "name": "Protein P0DJN9 from UniProtKB",
        "label": "Aerobic magnesium-protoporphyrin IX monomethyl ester [oxidative] cyclase",
        "subject": {
            "label": "Rubrivivax gelatinosus"
        }
    }
]


## Query the NeuroMorpho WebService

In [32]:
neuromorpho = sources['NeuroMorpho']

In [33]:
nmo_filters = {"species": "rat,mouse,human", "response_loc": ["_embedded", "neuronResources"]}

In [34]:
nmo_resources = forge.search(nmo_filters, db_source='NeuroMorpho')

/opt/miniconda3/envs/kgforge/lib/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'neuromorpho.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [35]:
import uuid

In [36]:
nmo_resources[2].bbpID = str(uuid.uuid4())

In [37]:
nmo_resources[2]

Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, _inner_sync=False, _links={'self': {'href': 'http://neuromorpho.org/api/neuron/id/100'}, 'measurements': {'href': 'http://neuromorpho.org/api/morphometry/id/100'}, 'persistence_vector': {'href': 'http://neuromorpho.org/api/pvec/id/100'}}, age_classification='young', age_scale='Month', archive='Turner', attributes='Diameter, 3D, Angles', bbpID='f947f690-a5ef-4ea4-aecf-89eb1145f400', brain_region=['hippocampus', 'CA1'], cell_type=['pyramidal', 'principal cell'], corrected_value=None, corrected_xy='133.0', corrected_z='400.0', deposition_date='2005-12-31', domain='Dendrites, Soma, No Axon', experiment_condition=['Control'], gender='Male/Female', magnification='100', max_age='8.0', max_weight='350.0', min_age='2.0', min_weight='200.0', neuron_id=100, neuron_name='n419', note='When originally released, this reconstruction had been incompletely processed, and this issue was fixed in release 6.3 (February

In [38]:
new_morphology = neuromorpho.map(nmo_resources[2], 'NeuronMorphology')

In [39]:
print(json.dumps(forge.as_jsonld(new_morphology), indent=4))

[
    {
        "@context": "https://bbp.neuroshapes.org",
        "@type": [
            "Dataset",
            "NeuronMorphology"
        ],
        "@id": "https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/neuromorpho/f947f690-a5ef-4ea4-aecf-89eb1145f400",
        "brainLocation": {
            "@type": "BrainLocation",
            "brainRegion": {
                "label": "hippocampus"
            }
        },
        "contribution": {
            "@type": "Contribution",
            "agent": {
                "@type": "Organization",
                "label": "George Mason University"
            }
        },
        "identifier": {
            "propertyID": "NeuroMorhoID",
            "value": 100
        },
        "archive": "Turner",
        "name": "n419",
        "generation": {
            "@type": "Generation",
            "activity": {
                "@type": "nsg:NeuronMorphologyReconstruction"
            }
        },
        "subject": {
            "@type"

In [40]:
format_file = neuromorpho.service.files_download['endpoint'] + "/{}/CNG version/{}.CNG.swc"

In [41]:
example_url = format_file.format(new_morphology[0].archive.lower(), new_morphology[0].name)

In [42]:
clean_name = ''.join(example_url.split('.')[:-2])+'.swc'

In [43]:
clean_name

'http://neuromorphoorg/dableFiles/turner/CNG version/n419.swc'

In [44]:
file_path = f"./downloaded/{clean_name.split('/')[-1]}"

In [45]:
file_path

'./downloaded/n419.swc'

In [46]:
neuromorpho._download_one(example_url, file_path)

/opt/miniconda3/envs/kgforge/lib/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'neuromorpho.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [47]:
neuromorpho.attach_file(new_morphology[0], file_path, content_type='application/swc')

In [48]:
print(new_morphology[0])

{
    id: https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/neuromorpho/f947f690-a5ef-4ea4-aecf-89eb1145f400
    type:
    [
        Dataset
        NeuronMorphology
    ]
    archive: Turner
    brainLocation:
    {
        type: BrainLocation
        brainRegion:
        {
            label: hippocampus
        }
    }
    contribution:
    {
        type: Contribution
        agent:
        {
            type: Organization
            label: George Mason University
        }
    }
    dateCreated:
    {
        type: xsd:dateTime
        value: 2005-12-31
    }
    description: Morphology of n419 obtained from NeuroMorpho API.
    distribution: LazyAction(operation=Store.upload, args=['./downloaded/n419.swc', 'application/swc'])
    generation:
    {
        type: Generation
        activity:
        {
            type: nsg:NeuronMorphologyReconstruction
        }
    }
    identifier:
    {
        propertyID: NeuroMorhoID
        value: 100
    }
    license:
    {
   

In [49]:
forge.validate(new_morphology[0], type_="NeuronMorphology")

<action> _validate_one
<succeeded> False
<error> ValidationError: resource has lazy actions which need to be executed before


In [50]:
# forge.register(new_morphology[0], schema_id="datashapes:neuronmorphology")

In [51]:
# new_morphology[0].brainLocation.brainRegion

In [52]:
# new_morphology[0].brainLocation.brainRegion = forge.resolve('hippocampal formation', scope='ontology', strategy='BEST_MATCH')

In [53]:
print(json.dumps(forge.as_jsonld(new_morphology[0]), indent=4))

ValueError: can't convert, resource contains LazyActions